In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture('../input_1080p.mp4')

_ , prev_frame = cap.read()
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#  Shi-Tomashi Corner Detection
mask = np.zeros_like(prev_frame_gray)
mask[400:600, 1000:1300] = 255
n_points = 20
kp_prev = cv2.goodFeaturesToTrack(prev_frame_gray, mask=mask.astype(np.uint8), maxCorners=n_points, qualityLevel=0.5, minDistance=50, blockSize=10)

mask = np.zeros_like(prev_frame)
color = np.random.randint(0, 255, (n_points, 3))

In [3]:
while(True):
      
    try:
        _, frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        #  Lucas-Kanade optical flows calculation
        kp, st, err = cv2.calcOpticalFlowPyrLK(prev_frame_gray, frame_gray, kp_prev, None, winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

        good_new = kp[st == 1]
        good_old = kp_prev[st == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            x1, y1 = new.ravel()
            x2, y2 = old.ravel()
            mask = cv2.line(mask, (int(x1),int(y1)), (int(x2),int(y2)), color[i].tolist(), 2)
            frame = cv2.circle(frame, (int(x1),int(y1)), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        img = cv2.resize(img, (1280, 720), interpolation=cv2.INTER_AREA)
  
        cv2.imshow('Video', img)

        prev_frame_gray = frame_gray.copy()
        kp_prev = good_new.reshape(-1, 1, 2)
    
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
    except:
        break
  
cv2.destroyAllWindows()
cap.release()

: 